In [1]:
import pandas as pd
import re
import preprocessor as p
import string
import emoji
import unicodedata
from nltk.corpus import stopwords
import json

## Importing JSON data

In [2]:
# Reading tweets from JSON file and ading them to Pandas Dataframe
tweets = pd.read_json('saver_output.json')

In [3]:
tweets.head(10)

,id,date,hashtags,text
0,1256010420462260224,2020-04-30 23:59:54+00:00,#allevamentiintensivi #coronavirus #Trumpvirus,Usa si fermano allevamentiintensivi e impianti...
1,1256010337557471232,2020-04-30 23:59:34+00:00,,Coronavirus Speranza firma decreto criteri mon...
2,1256010229252345856,2020-04-30 23:59:08+00:00,,La Lega occupa il Parlamento a rischio Coronav...
3,1256010127574028288,2020-04-30 23:58:44+00:00,#gb #boris #johnson #superato,gb boris johnson ampquotSuperato il picco di e...
4,1256010020208205824,2020-04-30 23:58:19+00:00,,Coronavirus il bilancio del 30 aprile record d...
5,1256009696428916736,2020-04-30 23:57:01+00:00,#Coronavirus #230mila #morti #mondo,Coronavirus pi di 230mila morti nel mondo Posi...
6,1256009535334035456,2020-04-30 23:56:23+00:00,#COVID19,COVID19 Esiste una teoria piuttosto seria seco...
7,1256009495605661696,2020-04-30 23:56:13+00:00,,Secondo tito boeri da cui ilsole24ore ha ripre...
8,1256009213307994112,2020-04-30 23:55:06+00:00,,La Lega occupa il Parlamento a rischio Coronav...
9,1256009157452365824,2020-04-30 23:54:53+00:00,,Coronavirus dagli assembramenti agli orari di ...


## Cleaning

### Preprocessor and emoji

In [4]:
# Setting preprocesor options
p.set_options(p.OPT.URL,p.OPT.MENTION)

In [5]:
# Function for cleaning with preprocessor
# It removes URLs and mentions
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [6]:
# Demojize
# Replaces emojis with text
def demojize(row):
    text = row['text']
    text = emoji.demojize(text)
    return text

In [7]:
# Applying previously defined functions
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)
tweets['text'] = tweets.apply(demojize, axis=1)

In [8]:
tweets.head(10)

,id,date,hashtags,text
0,1256010420462260224,2020-04-30 23:59:54+00:00,#allevamentiintensivi #coronavirus #Trumpvirus,Usa si fermano allevamentiintensivi e impianti...
1,1256010337557471232,2020-04-30 23:59:34+00:00,,Coronavirus Speranza firma decreto criteri mon...
2,1256010229252345856,2020-04-30 23:59:08+00:00,,La Lega occupa il Parlamento a rischio Coronav...
3,1256010127574028288,2020-04-30 23:58:44+00:00,#gb #boris #johnson #superato,gb boris johnson ampquotSuperato il picco di e...
4,1256010020208205824,2020-04-30 23:58:19+00:00,,Coronavirus il bilancio del 30 aprile record d...
5,1256009696428916736,2020-04-30 23:57:01+00:00,#Coronavirus #230mila #morti #mondo,Coronavirus pi di 230mila morti nel mondo Posi...
6,1256009535334035456,2020-04-30 23:56:23+00:00,#COVID19,COVID19 Esiste una teoria piuttosto seria seco...
7,1256009495605661696,2020-04-30 23:56:13+00:00,,Secondo tito boeri da cui ilsole24ore ha ripre...
8,1256009213307994112,2020-04-30 23:55:06+00:00,,La Lega occupa il Parlamento a rischio Coronav...
9,1256009157452365824,2020-04-30 23:54:53+00:00,,Coronavirus dagli assembramenti agli orari di ...


### Re

In [9]:
# Deleting duplicate tweets
tweets = tweets.drop_duplicates(subset='text')
tweets['data'] = tweets['text']

In [10]:
# Defining stopwords and adding a space before and after to exclude the case
# in which the stopword is cointained in a word
words = set(stopwords.words('italian'))
stopwords = [' ' + x + ' ' for x in words]

In [11]:
# Removing the tags
tweets.text = tweets.text.replace(
    "@[\w]*[_-]*[\w]*", " ", regex=True)
# Removing the spaces in excess
tweets.text = tweets.text.replace('\s+', ' ', regex=True)
# Removing the space at the beginning
tweets.text = tweets.text.replace('^ ', '', regex=True)
# Removing the space at the end
tweets.text = tweets.text.replace(' $', '', regex=True)
# To lowercase
tweets.text = tweets.text.apply(
    lambda x: x.lower())
tweets.text = tweets.text.replace('^', ' ', regex=True)
tweets.text = tweets.text.replace('$', ' ', regex=True)

# Removing stopwords
for word in stopwords:
    tweets.text = tweets.text.replace(word, ' ', regex=True)

# Removing the space at the beginning and at the end
tweets.text = tweets.text.apply(lambda x: x.strip())
# Removing empty tweets
tweets = tweets[tweets.text != '']

In [12]:
tweets.head(10)

,id,date,hashtags,text,data
0,1256010420462260224,2020-04-30 23:59:54+00:00,#allevamentiintensivi #coronavirus #Trumpvirus,usa fermano allevamentiintensivi impianti lavo...,Usa si fermano allevamentiintensivi e impianti...
1,1256010337557471232,2020-04-30 23:59:34+00:00,,coronavirus speranza firma decreto criteri mon...,Coronavirus Speranza firma decreto criteri mon...
2,1256010229252345856,2020-04-30 23:59:08+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
3,1256010127574028288,2020-04-30 23:58:44+00:00,#gb #boris #johnson #superato,gb boris johnson ampquotsuperato picco epidemi...,gb boris johnson ampquotSuperato il picco di e...
4,1256010020208205824,2020-04-30 23:58:19+00:00,,coronavirus bilancio 30 aprile record guariti ...,Coronavirus il bilancio del 30 aprile record d...
5,1256009696428916736,2020-04-30 23:57:01+00:00,#Coronavirus #230mila #morti #mondo,coronavirus pi 230mila morti mondo positivo pr...,Coronavirus pi di 230mila morti nel mondo Posi...
6,1256009535334035456,2020-04-30 23:56:23+00:00,#COVID19,covid19 esiste teoria piuttosto seria secondo ...,COVID19 Esiste una teoria piuttosto seria seco...
7,1256009495605661696,2020-04-30 23:56:13+00:00,,secondo tito boeri ilsole24ore ripreso relazio...,Secondo tito boeri da cui ilsole24ore ha ripre...
8,1256009213307994112,2020-04-30 23:55:06+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
9,1256009157452365824,2020-04-30 23:54:53+00:00,,coronavirus assembramenti orari punta fino con...,Coronavirus dagli assembramenti agli orari di ...


### Further cleaning

In [13]:
# Removing usernames in case preprocessor didn't work
def remove_mentions(text):
    return re.sub(r"(?:\@|https?\://)\S+", "", text)


# Remove punctuation in case Re missed something
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [14]:
# Removing mentions
tweets['text'] = tweets['text'].apply(remove_mentions)

# Removing punctuation
tweets['text'] = tweets['text'].apply(remove_punctuations)

In [15]:
tweets.head(10)

,id,date,hashtags,text,data
0,1256010420462260224,2020-04-30 23:59:54+00:00,#allevamentiintensivi #coronavirus #Trumpvirus,usa fermano allevamentiintensivi impianti lavo...,Usa si fermano allevamentiintensivi e impianti...
1,1256010337557471232,2020-04-30 23:59:34+00:00,,coronavirus speranza firma decreto criteri mon...,Coronavirus Speranza firma decreto criteri mon...
2,1256010229252345856,2020-04-30 23:59:08+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
3,1256010127574028288,2020-04-30 23:58:44+00:00,#gb #boris #johnson #superato,gb boris johnson ampquotsuperato picco epidemi...,gb boris johnson ampquotSuperato il picco di e...
4,1256010020208205824,2020-04-30 23:58:19+00:00,,coronavirus bilancio 30 aprile record guariti ...,Coronavirus il bilancio del 30 aprile record d...
5,1256009696428916736,2020-04-30 23:57:01+00:00,#Coronavirus #230mila #morti #mondo,coronavirus pi 230mila morti mondo positivo pr...,Coronavirus pi di 230mila morti nel mondo Posi...
6,1256009535334035456,2020-04-30 23:56:23+00:00,#COVID19,covid19 esiste teoria piuttosto seria secondo ...,COVID19 Esiste una teoria piuttosto seria seco...
7,1256009495605661696,2020-04-30 23:56:13+00:00,,secondo tito boeri ilsole24ore ripreso relazio...,Secondo tito boeri da cui ilsole24ore ha ripre...
8,1256009213307994112,2020-04-30 23:55:06+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
9,1256009157452365824,2020-04-30 23:54:53+00:00,,coronavirus assembramenti orari punta fino con...,Coronavirus dagli assembramenti agli orari di ...


In [16]:
# Remove accents in order to mitigate weird charachters in the final result
def strip_accents(row):
    text = row['text']
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

In [17]:
# Removing accent
tweets['text'] = tweets.apply(strip_accents, axis=1)

In [18]:
tweets.head(10)

,id,date,hashtags,text,data
0,1256010420462260224,2020-04-30 23:59:54+00:00,#allevamentiintensivi #coronavirus #Trumpvirus,usa fermano allevamentiintensivi impianti lavo...,Usa si fermano allevamentiintensivi e impianti...
1,1256010337557471232,2020-04-30 23:59:34+00:00,,coronavirus speranza firma decreto criteri mon...,Coronavirus Speranza firma decreto criteri mon...
2,1256010229252345856,2020-04-30 23:59:08+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
3,1256010127574028288,2020-04-30 23:58:44+00:00,#gb #boris #johnson #superato,gb boris johnson ampquotsuperato picco epidemi...,gb boris johnson ampquotSuperato il picco di e...
4,1256010020208205824,2020-04-30 23:58:19+00:00,,coronavirus bilancio 30 aprile record guariti ...,Coronavirus il bilancio del 30 aprile record d...
5,1256009696428916736,2020-04-30 23:57:01+00:00,#Coronavirus #230mila #morti #mondo,coronavirus pi 230mila morti mondo positivo pr...,Coronavirus pi di 230mila morti nel mondo Posi...
6,1256009535334035456,2020-04-30 23:56:23+00:00,#COVID19,covid19 esiste teoria piuttosto seria secondo ...,COVID19 Esiste una teoria piuttosto seria seco...
7,1256009495605661696,2020-04-30 23:56:13+00:00,,secondo tito boeri ilsole24ore ripreso relazio...,Secondo tito boeri da cui ilsole24ore ha ripre...
8,1256009213307994112,2020-04-30 23:55:06+00:00,,lega occupa parlamento rischio coronavirus via...,La Lega occupa il Parlamento a rischio Coronav...
9,1256009157452365824,2020-04-30 23:54:53+00:00,,coronavirus assembramenti orari punta fino con...,Coronavirus dagli assembramenti agli orari di ...


## Exporting

In [19]:
# Saving to CSV
tweets.to_csv('dataCleaned.csv')